# 爬蟲code_3

## 爬官方網站連結（website, yt, fb, ig）、臉書專頁（fb_url）、商品套組數（set_num）、每一組商品套組的價格（set_price）、第一個商品套組預計實現時間（time_expected）

In [ ]:
from google.colab import files
# 上傳csv檔案
uploaded = files.upload()

Saving all_projects_ended_info_預購式專案.csv to all_projects_ended_info_預購式專案.csv


In [9]:
# 將csv轉成data frame
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['all_projects_ended_info_預購式專案.csv']))
print(len(df))
df

962


,Unnamed: 0,title,author,category,money,time_left,url,content
0,0,361°龔俊同款軌跡系列連帽T及圓領短袖T預購 | Go For Joy 無畏變更,361度,時尚,"3,840,451",success,https://www.zeczec.com/projects/go-for-joy-361-t,首發9000件即秒殺的軌跡系列連帽T，嘖嘖正式展開預購，僅此一檔不要錯過！【 361°龔俊同...
1,1,【 聖保羅 I 餅 】好❤吃 ⇔ 好❤玩的伴手禮盒,聖保羅烘焙花園,飲食,"35,524",success,https://www.zeczec.com/projects/saintpaul-icake,小時候「吃」跟「玩」曾經畫上等號， 隨著成長的過程，我們似乎忘了好吃跟好玩其實是好朋友，I餅...
2,2,韓國 CUSVEN｜你的第一支頂級規格手錶｜超越期待的生活搭配用錶,Cusven_Taiwan,設計,"280,200",success,https://www.zeczec.com/projects/cusven,提供萬元內頂級規格生活手錶：義大利皮革、瑞士機芯、頂級藍寶石錶鏡韓國高人氣國名手錶新銳品牌 ...
3,3,釋放 50% 衣櫃空間，令人怦然心動的衣物收納神器「韓國 Ezam 魔法折疊衣架」,Neo Choice,設計,"1,476,715",success,https://www.zeczec.com/projects/ezam,專案已經於 2021/11/30 結束了。感謝各位贊助者支持，順利完成募資。我們已在愛料理上...
4,4,【台灣製 | 日虎CBD微膠囊舒麻枕】漢麻帶來舒壓入眠 × 慢彈記憶枕芯，每晚徹底放鬆，睡眠...,祥閎股份有限公司,設計,"44,100",success,https://www.zeczec.com/projects/cbd-x,添加CBD微膠囊記憶枕，舒緩一整天的壓力，帶給大家最放鬆的睡眠時光！在團隊努力下現在已經非常...
...,...,...,...,...,...,...,...,...
957,957,AICC雲端畫屏│顛覆傳統藝術鑑賞，是生活家也是創作家。,郭麥兒,科技,0,failure,https://www.zeczec.com/projects/aicccloud,一屏一木一世界，結合智能輪播，遠端操控等多種功能，畫屏內容隨心所欲。AICC雲端畫屏突破傳統...
958,958,廚房空氣清淨工程,張偉顥,飲食,"56,640",failure,https://www.zeczec.com/projects/peng-ren-qing-...,用地表最強的排油煙機，保護煮婦的健康。美味留在廚房，健康留在身上。只要一開起爐火，排油煙機就...
959,959,Aning DV 新型態 內建式 安全帽 機車 行車紀錄器 - gogoro 電動自行車 適用,PINJBAO,科技,"82,240",failure,https://www.zeczec.com/projects/aning-dv,新型態的行車紀錄器，透過各種發想來減少市面上行車紀錄器的使用不便，已提升更好、功能更強大的作...
960,960,我的行動暖氣圍巾,翁瑩純,設計,"5,900",failure,https://www.zeczec.com/projects/warmwarmbeeco,我是暖暖蜜蜂創辦人翁瑩純'， 2016年一月台灣特別寒冷，真的冷到寒風刺骨。台灣各地、台北山...


In [10]:
import requests
import json
from bs4 import BeautifulSoup

In [11]:
for i in range(0,5):
  print(df['title'][i],df['url'][i])

361°龔俊同款軌跡系列連帽T及圓領短袖T預購 | Go For Joy 無畏變更 https://www.zeczec.com/projects/go-for-joy-361-t
【 聖保羅 I 餅 】好❤吃 ⇔ 好❤玩的伴手禮盒 https://www.zeczec.com/projects/saintpaul-icake
韓國 CUSVEN｜你的第一支頂級規格手錶｜超越期待的生活搭配用錶 https://www.zeczec.com/projects/cusven
釋放 50% 衣櫃空間，令人怦然心動的衣物收納神器「韓國 Ezam 魔法折疊衣架」 https://www.zeczec.com/projects/ezam
【台灣製 | 日虎CBD微膠囊舒麻枕】漢麻帶來舒壓入眠 × 慢彈記憶枕芯，每晚徹底放鬆，睡眠的最佳享受！ https://www.zeczec.com/projects/cbd-x


In [12]:
all_data = []

for i in range(0,962):
    pj_dict = {}
    url = df['url'][i]
    user = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
    headers = {
        'User-Agent': user
    }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'html.parser')
      
    pj_dict['title'] = df['title'][i]
    # website
    web = 0
    web_result = soup.find_all(class_="ml2 relaitve dib tooltip lh-0 mid-gray tooltip-t")
    for w in web_result:
      if w['aria-label'] == '網站':
        web = 1
    pj_dict['website'] = web

    # fb&yt&ig
    fb = 0
    ig = 0
    yt = 0
    furl = ''
    social_result = soup.find_all(class_='ml2 lh-0 relaitve dib tooltip tooltip-t')
    for s in social_result:
      if s['aria-label'] == 'YouTube':
        yt = 1
      if s['aria-label'] == 'Facebook':
        fb = 1
        furl = s['href']
      if s['aria-label'] == 'Instagram':
        ig = 1
    if fb == 0:
      furl = 'None'
    pj_dict['yt'] = yt
    pj_dict['fb'] = fb
    pj_dict['ig'] = ig
    pj_dict['fb_url'] = furl

    # number of product set
    set_num = 0
    pd_result = soup.find_all(class_='w-100-l w-50-ns ph3 ph0-l flex-none self-start')
    for pd in pd_result:
      set_num += 1
    pj_dict['set_num'] = set_num

    # price of each product set
    price_list = []
    setp_result = soup.find_all(class_='black b f4')
    for sp in setp_result:
      if sp.text[1] == 'N':
        pstr = sp.text[4:-1]
        if ',' in pstr:
          pstr = pstr.replace(",","")
        price_list.append(int(pstr))
    pj_dict['set_price'] = price_list

    # time expected
    time_expt = ''
    time_result = soup.find_all(class_='mt3 gray tc ph2 f7 ba')
    for time in time_result:
      if time.text[0] == '預':
        time_expt = time.text[4:-2]
        break
      else:
        time_expt = 'None'
    pj_dict['time_expected'] = time_expt
    all_data.append(pj_dict)

In [13]:
import pandas as pd
df1 = pd.DataFrame(all_data)
df1

,title,website,yt,fb,ig,fb_url,set_num,set_price,time_expected
0,361°龔俊同款軌跡系列連帽T及圓領短袖T預購 | Go For Joy 無畏變更,0,0,1,1,https://www.facebook.com/361sportTaiwan/,5,"[2900, 1490, 2900, 850, 1500]",2022 年三月
1,【 聖保羅 I 餅 】好❤吃 ⇔ 好❤玩的伴手禮盒,1,0,1,1,https://www.facebook.com/sssaintpaul,2,"[388, 568]",2021 年十二月
2,韓國 CUSVEN｜你的第一支頂級規格手錶｜超越期待的生活搭配用錶,0,1,0,1,None,16,"[7750, 7750, 5190, 5650, 5650, 5190, 5190, 519...",2022 年一月
3,釋放 50% 衣櫃空間，令人怦然心動的衣物收納神器「韓國 Ezam 魔法折疊衣架」,0,0,1,1,https://www.facebook.com/neo.choice,50,"[365, 480, 550, 595, 698, 785, 798, 895, 968, ...",2022 年一月
4,【台灣製 | 日虎CBD微膠囊舒麻枕】漢麻帶來舒壓入眠 × 慢彈記憶枕芯，每晚徹底放鬆，睡眠...,1,0,1,0,https://www.facebook.com/citynoble/,5,"[2100, 1800, 4200, 6150, 10000]",2021 年十二月
...,...,...,...,...,...,...,...,...,...
957,AICC雲端畫屏│顛覆傳統藝術鑑賞，是生活家也是創作家。,0,0,0,1,None,4,"[12800, 14880, 27760, 51520]",2019 年四月
958,廚房空氣清淨工程,0,0,0,0,None,2,"[18880, 20800]",2019 年三月
959,Aning DV 新型態 內建式 安全帽 機車 行車紀錄器 - gogoro 電動自行車 適用,1,0,0,0,None,8,"[100, 5080, 3580, 6960, 5380, 10560, 20720, 30...",2019 年六月
960,我的行動暖氣圍巾,0,0,0,0,None,9,"[850, 1650, 4800, 9360, 700, 700, 700, 700, 1360]",2019 年四月


In [14]:
df1.to_csv("預購式專案_ended.csv")